# Python code for Chapter 2: Hospital Accessibility Catchment Areas as a Fuzzy Lattice Data Structure for toy example. Please note that the data used in the hospital catchment areas is not publically available and cannot be uploaded in this repository. Data ethics clearance number NAS003/2023.
 

# Importing all the relevant packages. This code was run on python 3.7.1

In [23]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import networkx as nx
import os
from functools import reduce

# Enable inline plotting
%matplotlib inline

# NB: The names in the structured nodes (1-9) are in the ordered format for V that the labelled states are first  i.e. grid_name={1,2,3,4,5,6,7,8,9}={v2,v7,v1,v3,v4,v5,v6,v8,v9}


# Import Datasets

In [ ]:
#Import Grid_Neighbours_Labels. 
# Grid_Name and Grid_Name2 indicates the the structural nodes adjacent to each other. 
# Absorbant_State indicates if grid_name is absorbant(1) or not and same for Absorbant_State2 for grid_name2. 
# Label_Val indicates the label associated with Grid_Name where for the toy example, P1 and P1 indicates the two POIs and U3-U9 indicates unlabelled.

# Define the base path
base_path = '//cenana01/testcode/pythonUserVenv/michelled/Virtual_Environment/PhD/Chapter 2/'
file_path = os.path.join(base_path, 'Grid_Neighbours_Labels.txt')

# Read the file
DF = pd.read_csv(file_path, sep='|')
print(DF.head())

   Grid_Name  Grid_Name2  Absorbant_State  Absorbant_State2 Label_val
0          1           1                1                 1        P1
1          3           1                0                 1        U3
2          4           1                0                 1        U4
3          6           1                0                 1        U6
4          2           2                1                 1        P2


In [3]:
# Degrees File. Distinct on grid_name and give the total number of neighbouring structural nodes(total) and if it is an absorbent state
# Define the base path
file_path2 = os.path.join(base_path, 'Grid_Degrees.txt')

# Read the file
DF2 = pd.read_csv(file_path2, sep='|')
print(DF2.head())

absorbant_States=sum(DF2.Absorbant_State)
nodes=len(DF2.index)

   grid_Name  Absorbant_State  Total
0          1                1      3
1          2                1      2
2          3                0      2
3          4                0      2
4          5                0      3


# Create Adjacency Matrix

In [4]:
#Using the information from DF create adjacency matrix 
df = pd.crosstab(DF.Grid_Name, DF.Grid_Name2)
idx = df.columns.union(df.index)
df = df.reindex(index = idx, columns=idx, fill_value=0)

#Create correct format in Links table (new adjacency matrix)
adjacency_matrix = np.matrix(df).astype(np.float64)
identity_abs = np.identity(absorbant_States)
#Absorbent states should only indicate 1 to own state, nullify absorbent states and replace top left hand corner of matrix with identity matrix
adjacency_matrix[0:absorbant_States]=0
#Replace the identity matrix in the top left hand corner of links matrix that all absorbent states can only move into themselves
adjacency_matrix[0:absorbant_States,0:absorbant_States]=identity_abs

#Links is now the adjacency matrix but all absorbent states only has a probability of 1 to stay in the same state with the rest 0
print(adjacency_matrix)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 1. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0.]]


# Create graph structure

In [5]:
df2 = np.diag(DF2.Total)

degree_matrix = np.matrix(df2)
#Replace the identity matrix in the top left hand corner of degree matrix 
degree_matrix[0:absorbant_States,0:absorbant_States]=identity_abs

# Verify dimension consistency
num_nodes_degree, _ = degree_matrix.shape
num_nodes_adjacency, _ = adjacency_matrix.shape

if num_nodes_degree != num_nodes_adjacency:
    print("Error: Degree and adjacency matrices have different dimensions.")

# Check the dimensions of the adjacency matrix
num_rows, num_cols = adjacency_matrix.shape

if num_rows != num_cols:
    print("Error: Adjacency matrix must be square.")
    # Handle the error appropriately

# Create a graph from the adjacency matrix
GM = nx.from_numpy_array(adjacency_matrix)

print(GM)

absorbent_states = list(range(absorbant_States))

Graph with 9 nodes and 14 edges


In [6]:
# Assign the degree information to the nodes
for i, node in enumerate(GM.nodes()):
    # Ensure that the index i is within bounds of the degree matrix
    if i < len(degree_matrix):
        # Use the node index directly to access the corresponding degree
        GM.nodes[node]['degree'] = degree_matrix[i]
    else:
        print(f"Index {i} is out of bounds for the degree matrix")

# Label proagation

In [7]:
def label_propagation_with_absorbent_states(G, absorbent_states, max_iter=100):
    """
    Label propagation algorithm implementation with absorbent states.
    
    Parameters:
    - G: NetworkX graph object.
    - absorbent_states: List of absorbent states (nodes).
    - max_iter: Maximum number of iterations.
    
    Returns:
    - A matrix where each row corresponds to a node and each column corresponds to an absorbent state.
    - Each element represents the probability of a node ending up in an absorbent state.
    """
    # Initialize probabilities matrix with zeros
    num_nodes = len(G)
    num_absorbent_states = len(absorbent_states)
    probabilities_matrix = np.zeros((num_nodes, num_absorbent_states))

    # Iterate until convergence or maximum iterations reached
    for _ in range(max_iter):
        prev_probabilities_matrix = probabilities_matrix.copy()
        
        # Update probabilities for each node
        for node in G.nodes():
            if node in absorbent_states:
                # If the node is an absorbent state, set its probability to 1
                absorbent_index = absorbent_states.index(node)
                probabilities_matrix[node][absorbent_index] = 1
            else:
                # Get neighbor probabilities
                neighbor_indices = [neighbor for neighbor in G.neighbors(node)]
                neighbor_probabilities = prev_probabilities_matrix[neighbor_indices].sum(axis=0)
                # If neighbors have probabilities, normalize and update current node's probabilities
                total_probability = neighbor_probabilities.sum()
                if total_probability > 0:
                    probabilities_matrix[node] = neighbor_probabilities / total_probability
        
        # Check for convergence
        if np.allclose(prev_probabilities_matrix, probabilities_matrix):
            break
    
    return probabilities_matrix

In [8]:
# Example usage:
if __name__ == "__main__":
    # Create a random graph (you can use any networkx graph)
    G = GM

    # Apply label propagation algorithm with absorbent states
    L = label_propagation_with_absorbent_states(G, absorbent_states)

    # Print the probabilities matrix
    print(L)

[[1.         0.        ]
 [0.         1.        ]
 [0.72413818 0.27586182]
 [0.82758685 0.17241315]
 [0.44827541 0.55172459]
 [0.62069034 0.37930966]
 [0.65517123 0.34482877]
 [0.37930947 0.62069053]
 [0.5172425  0.4827575 ]]


# Drivetimes: Please note for the toy example the same nodes are indicated for drive-times 5, 10 and 15minutes but for real-world applications like the hospitals more nodes will be included as the drive times extend. The nodes captured in each drive time layer needs to be indicated in the Grids_to_Drivetimes_5min, Grids_to_Drivetimes_10min and Grids_to_Drivetimes_15min txt files.

In [9]:
# The Grid_Name column indicates the grid that is allocated in the drive time circle of Store_Grid. I.e. grid names 1 and 2 are the only grids with a POI and which have a drivetime circle associated with it.
file_path3 = os.path.join(base_path, 'Grids_to_Drivetimes_5min.txt')

# Read the file
DT5 = pd.read_csv(file_path3, sep='|')
print(DT5)

   grid_name  store_grid
0          1           1
1          4           1
2          6           1
3          7           1
4          2           2
5          5           2
6          6           2
7          8           2


In [10]:
# The Grid_Name column indicates the grid that is allocated in the drive time circle of Store_Grid. I.e. grid names 1 and 2 are the only grids with a POI and which have a drivetime circle associated with it.
file_path4 = os.path.join(base_path, 'Grids_to_Drivetimes_10min.txt')

# Read the file
DT10 = pd.read_csv(file_path4, sep='|')
print(DT10)

   grid_name  store_grid
0          1           1
1          4           1
2          6           1
3          7           1
4          2           2
5          5           2
6          6           2
7          8           2


In [11]:
# The Grid_Name column indicates the grid that is allocated in the drive time circle of Store_Grid. I.e. grid names 1 and 2 are the only grids with a POI and which have a drivetime circle associated with it.
file_path5 = os.path.join(base_path, 'Grids_to_Drivetimes_15min.txt')

# Read the file
DT15 = pd.read_csv(file_path5, sep='|')
print(DT15)

   grid_name  store_grid
0          1           1
1          4           1
2          6           1
3          7           1
4          2           2
5          5           2
6          6           2
7          8           2


In [12]:
#5 minute drive time mappings
#Create and indicator matrix with the columns the POIs and the rows indicating which nodes are in the drive-times of those POIs
n_cols, n_rows = absorbant_States+1, nodes+1
Dist5 = np.zeros((n_rows, n_cols), dtype=np.int32)

Dist5[0,:] = np.arange(n_cols)
Dist5[:,0] = np.arange(n_rows)

d_selection5 = pd.DataFrame(DT5, columns = ['grid_name','store_grid'])

for i in range(len(d_selection5)):
    Dist5[d_selection5.loc[i, "grid_name"],d_selection5.loc[i, "store_grid"]]=1

Dist5=pd.DataFrame(Dist5)
Dist5 = Dist5.iloc[1: , 1:]
Dist5=pd.DataFrame(Dist5)
print(Dist5)

myArray5 = np.where(Dist5==0, Dist5, L)
myArray5[0:absorbant_States,0:absorbant_States]=identity_abs
res5 = myArray5/myArray5.sum(axis=1)[:,None]
res5=pd.DataFrame(res5)
res5=res5.fillna(0)

   1  2
1  1  0
2  0  1
3  0  0
4  1  0
5  0  1
6  1  1
7  1  0
8  0  1
9  0  0


In [13]:
#10 minute drive time mappings
#Create and indicator matrix with the columns the POIs and the rows indicating which nodes are in the drive-times of those POIs
n_cols, n_rows = absorbant_States+1, nodes+1
Dist10 = np.zeros((n_rows, n_cols), dtype=np.int32)

Dist10[0,:] = np.arange(n_cols)
Dist10[:,0] = np.arange(n_rows)

d_selection10 = pd.DataFrame(DT10, columns = ['grid_name','store_grid'])

for i in range(len(d_selection10)):
    Dist10[d_selection10.loc[i, "grid_name"],d_selection10.loc[i, "store_grid"]]=1

Dist10=pd.DataFrame(Dist10)
Dist10 = Dist10.iloc[1: , 1:]
Dist10=pd.DataFrame(Dist10)
print(Dist10)

myArray10 = np.where(Dist10==0, Dist10, L)
myArray10[0:absorbant_States,0:absorbant_States]=identity_abs
res10 = myArray10/myArray10.sum(axis=1)[:,None]
res10=pd.DataFrame(res10)
res10=res10.fillna(0)

   1  2
1  1  0
2  0  1
3  0  0
4  1  0
5  0  1
6  1  1
7  1  0
8  0  1
9  0  0


In [14]:
#15 minute drive time mappings
#Create and indicator matrix with the columns the POIs and the rows indicating which nodes are in the drive-times of those POIs
n_cols, n_rows = absorbant_States+1, nodes+1
Dist15 = np.zeros((n_rows, n_cols), dtype=np.int32)

Dist15[0,:] = np.arange(n_cols)
Dist15[:,0] = np.arange(n_rows)

d_selection15 = pd.DataFrame(DT15, columns = ['grid_name','store_grid'])

for i in range(len(d_selection15)):
    Dist15[d_selection15.loc[i, "grid_name"],d_selection15.loc[i, "store_grid"]]=1

Dist15=pd.DataFrame(Dist15)
Dist15 = Dist15.iloc[1: , 1:]
Dist15=pd.DataFrame(Dist15)
print(Dist15)

myArray15 = np.where(Dist15==0, Dist15, L)
myArray15[0:absorbant_States,0:absorbant_States]=identity_abs
res15 = myArray10/myArray15.sum(axis=1)[:,None]
res15=pd.DataFrame(res15)
res15=res15.fillna(0)

   1  2
1  1  0
2  0  1
3  0  0
4  1  0
5  0  1
6  1  1
7  1  0
8  0  1
9  0  0


# Supply and demand to calculate supply-demand ratio and accessibility

In [15]:
#The Demand for each grid in the toy example is 1
file_path6 = os.path.join(base_path, 'Grid_Demand.txt')

# Read the file
Grid_Demand = pd.read_csv(file_path6, sep='|')

# Convert grid_name to numeric if it's stored as string
Grid_Demand['grid_name'] = Grid_Demand['grid_name'].astype(int)
# Now sort
Grid_Demand.sort_values('grid_name', inplace=True)
Grid_Demand.reset_index(drop=True, inplace=True)
print(Grid_Demand)

   grid_name  Total
0          1      1
1          2      1
2          3      1
3          4      1
4          5      1
5          6      1
6          7      1
7          8      1
8          9      1


In [16]:
# The supply for the toy example for P1 and P2 is 4
file_path7 = os.path.join(base_path, 'Grid_Supply.txt')

# Read the file
Grid_Supply = pd.read_csv(file_path7, sep='|')
print(Grid_Supply)

   grid_name  total_sales
0          1            4
1          2            4


In [17]:
#Calculate the total demand per grid based on the drive time rings: Demand at 5 minute drive-times
Grid_Demand=pd.DataFrame(Grid_Demand)
Grid_T=Grid_Demand.Total
res5_T=res5.T

Grid_Demand_5min=np.multiply(res5_T,Grid_T)
Grid_Demand_5min=Grid_Demand_5min.T

Total_Grid_Demand_5min=round(Grid_Demand_5min.sum(axis='rows'),2)
Total_Grid_Demand_5min=pd.DataFrame(Total_Grid_Demand_5min)
Total_Grid_Demand_5min.rename(columns={0:'Total_Grid_5min'})
Total_Grid_Demand_5min.insert(0, 'grid_name', range(1, 1 + len(Total_Grid_Demand_5min)))
Total_Grid_Demand_5min.rename(columns={0:'Total_Grid_5min'})

,grid_name,Total_Grid_5min
0,1,3.62
1,2,3.38


In [18]:
#Calculate the total demand per grid based on the drive time rings: Demand at 10 minute drive-times
Grid_Demand=pd.DataFrame(Grid_Demand)
Grid_T=Grid_Demand.Total
res10_T=res10.T

Grid_Demand_10min=np.multiply(res10_T,Grid_T)
Grid_Demand_10min=Grid_Demand_10min.T
Total_Grid_Demand_10min=round(Grid_Demand_10min.sum(axis='rows'),2)
Total_Grid_Demand_10min=pd.DataFrame(Total_Grid_Demand_10min)
Total_Grid_Demand_10min.rename(columns={0:'Total_Grid_10min'})
Total_Grid_Demand_10min.insert(0, 'grid_name', range(1, 1 + len(Total_Grid_Demand_10min)))
Total_Grid_Demand_10min.rename(columns={0:'Total_Grid_10min'})

,grid_name,Total_Grid_10min
0,1,3.62
1,2,3.38


In [19]:
#Calculate the total demand per grid based on the drive time rings: Demand at 15 minute drive-times
Grid_Demand=pd.DataFrame(Grid_Demand)
Grid_T=Grid_Demand.Total
res15_T=res15.T

Grid_Demand_15min=np.multiply(res15_T,Grid_T)
Grid_Demand_15min=Grid_Demand_15min.T
Total_Grid_Demand_15min=round(Grid_Demand_15min.sum(axis='rows'),2)
Total_Grid_Demand_15min=pd.DataFrame(Total_Grid_Demand_15min)
Total_Grid_Demand_15min.rename(columns={0:'Total_Grid_15min'})
Total_Grid_Demand_15min.insert(0, 'grid_name', range(1, 1 + len(Total_Grid_Demand_15min)))
Total_Grid_Demand_15min.rename(columns={0:'Total_Grid_15min'})

,grid_name,Total_Grid_15min
0,1,3.62
1,2,3.38


In [20]:
# merge all drive times together and supply
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['grid_name'],how='outer'), (Grid_Supply,Total_Grid_Demand_5min,Total_Grid_Demand_10min,Total_Grid_Demand_15min))        
df_merged=pd.DataFrame(df_merged)
df_merged.set_axis(['GRID_NAME', 'Total_Sales','Total_Demand_5min','Total_Demand_10min','Total_Demand_15min'], axis='columns', inplace=True)
print(df_merged)

   GRID_NAME  Total_Sales  Total_Demand_5min  Total_Demand_10min  \
0          1            4               3.62                3.62   
1          2            4               3.38                3.38   

   Total_Demand_15min  
0                3.62  
1                3.38  


In [ ]:
#Add Supply-Demand Ratio for each drive-time. Since for the toy example all drivetimes are the same, the supply-demand ratio is also the same for 5,10 and 15min
df_merged['Total_Sales'].fillna(0, inplace=True)
df_merged['Total_Demand_5min'].fillna(0, inplace=True)

df_merged['Total_Sales'] = pd.to_numeric(df_merged['Total_Sales'], errors='coerce')
df_merged['Total_Demand_5min'] = pd.to_numeric(df_merged['Total_Demand_5min'], errors='coerce')

df_merged['AR_5min']=df_merged['Total_Sales']/df_merged['Total_Demand_5min']
df_merged['AR_10min']=df_merged['Total_Sales']/df_merged['Total_Demand_10min']
df_merged['AR_15min']=df_merged['Total_Sales']/df_merged['Total_Demand_15min']

df_merged=pd.DataFrame(df_merged)
df_merged.fillna(0, inplace=True)
df_merged.sort_values(['GRID_NAME'], inplace=True)
df_merged = df_merged.round(2)

print(df_merged)

   GRID_NAME  Total_Sales  Total_Demand_5min  Total_Demand_10min  \
0          1            4               3.62                3.62   
1          2            4               3.38                3.38   

   Total_Demand_15min  AR_5min  AR_10min  AR_15min  
0                3.62     1.10      1.10      1.10  
1                3.38     1.18      1.18      1.18  


# Calculate the accessibility per grid

In [ ]:
Accessibility_5min = np.matmul(Dist5,df_merged.AR_5min)
Accessibility_10min = np.matmul(Dist10,df_merged.AR_10min)
Accessibility_15min = np.matmul(Dist15,df_merged.AR_15min)

Accessibility = [Accessibility_5min,Accessibility_10min,Accessibility_15min]
Accessibility=pd.DataFrame(Accessibility)
Accessibility=Accessibility.T
Accessibility.set_axis(['Accessibility_5min', 'Accessibility_10min', 'Accessibility_15min'], axis='columns', inplace=True)   

# Round all values to 2 decimal places
Accessibility = Accessibility.round(2)

print("Updated Accessibility DataFrame:")
print(Accessibility)
#The accessibility for grid v5 which in the ordered set V={v2,v7,v1,v3,v4,v5,v6,v8,v9} is grid_name 6 is 2.28


Updated Accessibility DataFrame:
   Accessibility_5min  Accessibility_10min  Accessibility_15min
1                1.10                 1.10                 1.10
2                1.18                 1.18                 1.18
3                0.00                 0.00                 0.00
4                1.10                 1.10                 1.10
5                1.18                 1.18                 1.18
6                2.28                 2.28                 2.28
7                1.10                 1.10                 1.10
8                1.18                 1.18                 1.18
9                0.00                 0.00                 0.00
